<a href="https://colab.research.google.com/github/luiscaballeroramos/DeCoder/blob/EC3_1_8_DesignOfJoints/DeCoder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 0. Preliminary Tasks

Install and import PyPDF2 library to open and read the PDF file containing the code

In [6]:
!pip install PyPDF2
import PyPDF2

Clone the GitHub repository containing the PDF file with code

In [7]:
# Clone the entire repo.
!git clone -l -s -b EC3_1_8_DesignOfJoints https://github.com/luiscaballeroramos/DeCoder.git DeCoder

import DeCoder_Auxiliary as aux
aux.Luis()

fatal: destination path 'DeCoder' already exists and is not an empty directory.


ModuleNotFoundError: ignored

Import libraries used in this tool

In [ ]:
import numpy as np
import regex as re
from tabulate import tabulate

# 1. Read PDF

Read the .pdf file with PyPF2 library

In [ ]:
pdfFile=open('DeCoder/EC 1993.1.8.2005-1.pdf','rb')
pdfReader=PyPDF2.PdfFileReader(pdfFile)

In [ ]:
numPages=pdfReader.numPages
print(numPages)

Get the pages content as dictionary (pages) with:
* key = page number (pageNum)
* value = page content as string (pageContent)



In [ ]:
pages={}
pageContent=''
for pageNum in range(0,numPages):
  pdfPage=pdfReader.getPage(pageNum)
  pageContent=pdfPage.extractText()
  pages[pageNum]=pageContent
  pass

# 2. Table Of Contents (TOC)
2.0. TOC Definition (settings)

2.1. TOC Location (pages)

2.2. TOC Contents (toc's)

# 2.0. TOC Definition

Set TOC identifiers, keywords that could be used to refering TOC

In [ ]:
TOCidentifiers=['contents','Contents',
                 'table of contents','Table of contents','Table Of Contents']

Description of toc elements by regex expressions 

In [ ]:
refidRegex='\d'
refsepRegex='\.'
refsep='.'
namRegex='[\s\w]*'
sepRegex='\.{2,}'
numRegex='\d+'

In [ ]:




TOC_EC3=TOC(refidRegex,refsepRegex,refsep,namRegex,sepRegex,numRegex)

In [ ]:
def join(l, sep):
    out_str = ''
    for i, el in enumerate(l):
        out_str += '{}{}'.format(el, sep)
    return out_str




# 2.1. TOC Location

TOC identifiers apearances in each document page 

In [ ]:
TOCcoincidences={}
TOCtotalCoincidences={}
for pageNum, pageContent in pages.items():
  aux=[]
  for TOCidentifier in TOCidentifiers:
    aux.append(pageContent.count(TOCidentifier))
    pass
  TOCcoincidences[pageNum]=aux
  TOCtotalCoincidences[pageNum]=sum(aux)
  pass

Total of TOC identifiers apearances in PDF document

In [ ]:
sumTOCTotalCoincidences=sum(TOCtotalCoincidences.values())
print('Number of TOC indentifiers: {0}'.format(sumTOCTotalCoincidences))

Max TOC identifiers apearances in one page and the page where this happens

In [ ]:
maxTOCTotalCoincidences=max(TOCtotalCoincidences.values())
maxTOCTotalCoincidencesPages = [key  for (key, value) in TOCtotalCoincidences.items() if value == maxTOCTotalCoincidences]
print('Number of Max TOC indentifiers in One Page: {0}'.format(maxTOCTotalCoincidences))
print('Pages with Max TOC indentifiers in One Page: {0}'.format(maxTOCTotalCoincidencesPages))

Pages where toc pattern (Regex) is detected

In [ ]:
tocCoincidences={}
for pageNum, pageContent in pages.items():
  tocCoincidences[pageNum]=len(re.findall(TOC_EC3.tocRegex,pages[pageNum]))
  pass

In [ ]:
noNulltocCoincidencesPages = [key  for (key, value) in tocCoincidences.items() if value != 0]
print('Pages with toc Regex: {0}'.format(noNulltocCoincidencesPages))

In [ ]:
TOCpages=noNulltocCoincidencesPages

# 2.2. TOC Contents

Look for complete tocRegex in TOCpages

In [ ]:
pattern = re.compile(TOC_EC3.tocRegex)
for page in TOCpages:
  for match in pattern.finditer(pages[page]):
    [split1,split2]=re.split(TOC_EC3.sepRegex,match.group())
    ref=re.search(TOC_EC3.refRegex,split1).group()
    split1=re.sub(TOC_EC3.refRegex,'',split1)
    nam=re.search(TOC_EC3.namRegex,split1).group()
    split1=re.sub(TOC_EC3.namRegex,'',split1)
    num=re.search(TOC_EC3.numRegex,split2).group()
    beginLocation=PDFlocation(num)
    refsep=TOC_EC3.refsep
    toc_EC3=toc(ref,refsep,nam,beginLocation)
    TOC_EC3.addToc(toc_EC3)
    pass
  pass
TOC_EC3.tocList.sort(key=lambda x: x.ref, reverse=False)
TOC_EC3.printTOC()

Complete with references that can be deduced to exist in TOC

In [ ]:
refLeft=[]
for toc in TOC_EC3.tocList:
  levels=re.split(TOC_EC3.refsepRegex,toc.ref)
  levels=list(filter(lambda item: item, levels))
  levels=list(map(int,levels))
  for level in range(toc.level+1):
    numLevel=levels[level]
    #if numLevel=1 there is no previous ref in this level
    if numLevel!=1:
      #find the first part of the ref at the levels>0
      if level!=0:
        prevref=''
        for i in range(level):
          prevref=prevref+str(levels[i])+TOC_EC3.refsep
          pass
      else:
        prevref=''
        pass
      #add to refLeft the lefts references
      for i in range(1,numLevel+1):
        refLeft.append(prevref+str(i)+TOC_EC3.refsep)
        pass
      pass
    pass
  pass
refLeft=list(set(refLeft))
refLeft.sort()
print(refLeft)
len(refLeft)

for ref in refLeft:
  refsep=TOC_EC3.refsep
  toc_EC3=toc(ref,refsep,nam)
  TOC_EC3.addToc(toc_EC3)
  pass
TOC_EC3.tocList.sort(key=lambda x: x.ref, reverse=False)
TOC_EC3.printTOC()

Complete TOC with references deduced from TOC's complete references

In [ ]:
pattern = re.compile(TOC_EC3.refRegex)
refList=[]
for page in pages:
  for match in pattern.finditer(pages[page]):
    refList.append(match.group())
    pass
  pass

refList=list(set(refList))
refList.sort()
print(refList)
len(refList)

# Close PDF

In [ ]:
pdfFile.close()